In [ ]:
library(data.table)
library(tidyverse)
library(gridExtra)

### 0 - Data Ready

In [ ]:
vafs <- fread("/home/jusset/vaf_clean/data/vaf_ready.csv") %>% select(-V1)

In [ ]:
saveRDS(vafs %>% filter(tier == "PANEL") %>% arrange(gene) %>% pull(gene) %>% unique(), 
        file = "/home/jusset/vaf_clean/data/panel_gene.Rds")

### 1 - Prep RNA VAF
- Remove subclonal variants
- Require purple_cn to be greater than 1
- Threshold purple allele frequency and RNA VAF estimate to be max 1

In [ ]:
vaf_ready <- 
vafs %>% 
  filter( subclonal < .1, purple_cn > .5, !is.na(purple_af) ) %>% 
  mutate( purple_af_mx = ifelse(biallelic, 1, purple_af)) %>% #### changed to biallelic
  mutate( rna_vaf_est = ifelse(rna_vaf/purple_af_mx > 1, 1, rna_vaf/purple_af_mx))

### 2 - Aggregations at Gene-level
- Aggregate sample and gene level aggregations 

In [ ]:
lets_go <- 
vaf_ready %>%
  group_by(sample, gene) %>% 
  summarise(
    rna_vaf = sum(rna_vaf_est*rna_dp)/sum(rna_dp),
    rna_dp = sum(rna_dp),
    rna_dp_adj = sum(rna_dp * purple_af_mx),
    purple_af_mn = mean(purple_af_mx),
    vafs = n(),
    biallelic = paste0(unique(biallelic), collapse = ","), 
    afs = paste0(purple_af, collapse = ",")  
  ) %>% ungroup() 

### 3 - Send it

In [ ]:
fwrite(lets_go, "/home/jusset/vaf_clean/data/vaf_ready.txt")